In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt 
from matplotlib import pyplot as plt
import pandas as pd
import math
import sklearn.metrics as sk

In [ ]:
class CLF:
  def __init__(self):
    (self.x_train, self.y_train), (self.x_test, self.y_test) = tf.keras.datasets.mnist.load_data()
    self.x_train = [np.round(1.0*i/256) for i in self.x_train] 
    self.x_test = [np.round(1.0*i/256) for i in self.x_test]


    self.PX0Y = np.zeros((10,4,28,28))
    self.PX1Y = np.zeros((10,4,28,28))

    self.sp_num = 1
    self.sp_denom = 2


  def compile(self):

    for digit in range(10):


      temp_train = []

      for i in range(len(self.y_train)):
        if self.y_train[i] == digit:
          temp_train.append(self.x_train[i])
      
      

      for i in range(28):
        for j in range(28):
          x_0_00_count = 0 #Left and Top format
          x_0_01_count = 0
          x_0_10_count = 0
          x_0_11_count = 0

          y_0_00_count = 0 # These are the denominator terms
          y_0_01_count = 0
          y_0_10_count = 0
          y_0_11_count = 0

          x0_count = 0

          if (i-1 > -1 and j-1 > -1):

            for k in range(len(temp_train)):
              if (temp_train[k][i][j-1] == 0 and temp_train[k][i-1][j] == 0):
                y_0_00_count += 1
                if (temp_train[k][i][j] == 0):
                  x_0_00_count += 1

              elif (temp_train[k][i][j-1] == 0 and temp_train[k][i-1][j] == 1):
                y_0_01_count += 1
                if (temp_train[k][i][j] == 0):
                  x_0_01_count += 1

              elif (temp_train[k][i][j-1] == 1 and temp_train[k][i-1][j] == 0):
                y_0_10_count += 1
                if (temp_train[k][i][j] == 0):
                  x_0_10_count += 1

              elif (temp_train[k][i][j-1] == 1 and temp_train[k][i-1][j] == 1):
                y_0_11_count += 1
                if (temp_train[k][i][j] == 0):
                  x_0_11_count += 1
              
            self.PX0Y[digit][0][i][j] = math.log((self.sp_num + x_0_00_count) / (self.sp_denom + y_0_00_count))
            self.PX1Y[digit][0][i][j] = math.log(1 - ((self.sp_num + x_0_00_count) / (self.sp_denom + y_0_00_count)))
          
            self.PX0Y[digit][1][i][j] = math.log((self.sp_num + x_0_01_count) / (self.sp_denom + y_0_01_count))
            self.PX1Y[digit][1][i][j] = math.log(1 - ((self.sp_num + x_0_01_count) / (self.sp_denom + y_0_01_count)))

            self.PX0Y[digit][2][i][j] = math.log((self.sp_num + x_0_10_count) / (self.sp_denom + y_0_10_count))
            self.PX1Y[digit][2][i][j] = math.log(1 - ((self.sp_num + x_0_10_count) / (self.sp_denom + y_0_10_count)))

            self.PX0Y[digit][3][i][j] = math.log((self.sp_num + x_0_11_count) / (self.sp_denom + y_0_11_count))
            self.PX1Y[digit][3][i][j] = math.log(1 - ((self.sp_num + x_0_11_count) / (self.sp_denom + y_0_11_count)))
          


          else:

            for k in range(len(temp_train)):
              if temp_train[k][i][j] == 0:
                x0_count += 1
            
            for l in range(4):
              self.PX0Y[digit][l][i][j] = math.log((self.sp_num + x0_count) / (self.sp_denom + len(temp_train)))
              self.PX1Y[digit][l][i][j] = math.log(1 - ((self.sp_num + x0_count) / (self.sp_denom + len(temp_train))))



  def predict(self, test_image):

    test_image = np.round(1.0*test_image/256)

    digit_sum = []

    for digit in range(10):
      temp_sum = 0

      for i in range(28):
        for j in range(28):

          if (i-1 > -1 and j-1 > -1):

            if (test_image[i][j-1] == 0 and test_image[i-1][j] == 0):
              if(test_image[i][j] == 0):
                temp_sum += self.PX0Y[digit][0][i][j]
              else:
                temp_sum += self.PX1Y[digit][0][i][j]

            if (test_image[i][j-1] == 0 and test_image[i-1][j] == 1):
              if(test_image[i][j] == 0):
                temp_sum += self.PX0Y[digit][1][i][j]
              else:
                temp_sum += self.PX1Y[digit][1][i][j]

            if (test_image[i][j-1] == 1 and test_image[i-1][j] == 0):
              if(test_image[i][j] == 0):
                temp_sum += self.PX0Y[digit][2][i][j]
              else:
                temp_sum += self.PX1Y[digit][2][i][j]

            if (test_image[i][j-1] == 1 and test_image[i-1][j] == 1):
              if(test_image[i][j] == 0):
                temp_sum += self.PX0Y[digit][3][i][j]
              else:
                temp_sum += self.PX1Y[digit][3][i][j]

          else: 
            if(test_image[i][j] == 0):
              temp_sum += self.PX0Y[digit][0][i][j]
            else: 
              temp_sum += self.PX1Y[digit][0][i][j]


      digit_sum.append(temp_sum)

    prediction = np.argmax(digit_sum)
    return prediction


In [ ]:
clf = CLF()
clf.compile()

In [ ]:
(x_tr, y_tr), (x_t, y_t) = tf.keras.datasets.mnist.load_data()
y_pred = []


for image in x_t:
  y_pred.append(clf.predict(image))


print(f'The Error is {1 - sk.accuracy_score(y_t, y_pred)}')
print(f'The accuracy is {sk.accuracy_score(y_t, y_pred)}')